In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from geopy.geocoders import Nominatim

df = pd.read_csv("Airplane_Crashes_and_Fatalities_Since_1908_t0_2023.csv", encoding='latin-1')
df.head()

,Date,Time,Location,Operator,Flight #,Route,AC Type,Registration,cn/ln,Aboard,Aboard Passangers,Aboard Crew,Fatalities,Fatalities Passangers,Fatalities Crew,Ground,Summary
0,9/17/1908,17:18,"Fort Myer, Virginia",Military - U.S. Army,NaN,Demonstration,Wright Flyer III,NaN,1,2.0,1.0,1.0,1.0,1.0,0.0,0.0,"During a demonstration flight, a U.S. Army fly..."
1,9/7/1909,NaN,"Juvisy-sur-Orge, France",NaN,NaN,Air show,Wright Byplane,SC1,NaN,1.0,0.0,1.0,1.0,0.0,0.0,0.0,Eugene Lefebvre was the first pilot to ever be...
2,7/12/1912,6:30,"Atlantic City, New Jersey",Military - U.S. Navy,NaN,Test flight,Dirigible,NaN,NaN,5.0,0.0,5.0,5.0,0.0,5.0,0.0,First U.S. dirigible Akron exploded just offsh...
3,8/6/1913,NaN,"Victoria, British Columbia, Canada",Private,NaN,NaN,Curtiss seaplane,NaN,NaN,1.0,0.0,1.0,1.0,0.0,1.0,0.0,The first fatal airplane accident in Canada oc...
4,9/9/1913,18:30,Over the North Sea,Military - German Navy,NaN,NaN,Zeppelin L-1 (airship),NaN,NaN,20.0,NaN,NaN,14.0,NaN,NaN,0.0,The airship flew into a thunderstorm and encou...


In [3]:
df.shape

(4998, 17)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4998 entries, 0 to 4997
Data columns (total 17 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Date                   4998 non-null   object 
 1   Time                   3486 non-null   object 
 2   Location               4994 non-null   object 
 3   Operator               4988 non-null   object 
 4   Flight #               1329 non-null   object 
 5   Route                  4222 non-null   object 
 6   AC Type                4983 non-null   object 
 7   Registration           4724 non-null   object 
 8   cn/ln                  4330 non-null   object 
 9   Aboard                 4980 non-null   float64
 10  Aboard Passangers      4769 non-null   float64
 11  Aboard Crew            4772 non-null   float64
 12  Fatalities             4990 non-null   float64
 13  Fatalities Passangers  4756 non-null   float64
 14  Fatalities Crew        4757 non-null   float64
 15  Grou

In [5]:
df.isnull().sum()

Date                        0
Time                     1512
Location                    4
Operator                   10
Flight #                 3669
Route                     776
AC Type                    15
Registration              274
cn/ln                     668
Aboard                     18
Aboard Passangers         229
Aboard Crew               226
Fatalities                  8
Fatalities Passangers     242
Fatalities Crew           241
Ground                     42
Summary                    64
dtype: int64

In [6]:
# Convert data types if needed (e.g., convert 'Date' column to datetime)
df['Date'] = pd.to_datetime(df['Date'])


In [7]:
df['Aboard'].fillna(df['Aboard'].median(), inplace=True)
df['Operator'].fillna('Unknown', inplace=True)


In [8]:
df['Operator'] = df['Operator'].str.strip().str.title()


In [9]:
#  identify and handle outliers
max_fatalities = df['Fatalities'].quantile(0.99)
df['Fatalities'] = df['Fatalities'].clip(upper=max_fatalities)

In [10]:
# Check for data integrity issues, for example, by verifying Aboard = Aboard Passengers + Aboard Crew
df['Aboard_Check'] = df['Aboard Passangers'] + df['Aboard Crew']
integrity_check = df[df['Aboard'] != df['Aboard_Check']]

In [11]:
# Resolve data integrity issues
# For example, update 'Aboard' column with the sum of passengers and crew
df['Aboard'] = df['Aboard Passangers'] + df['Aboard Crew']

In [12]:
df['Location'] = df['Location'].str.title()
df['Route']= df['Route'].str.title()
df['AC Type'] = df['AC Type'].str.title()
df['Registration']= df['Registration'].str.title()

In [13]:
# Check for and remove duplicate records
df_duplicates = df[df.duplicated()]
df = df.drop_duplicates()

In [14]:
#Delete all Nan Flights
df = df[pd.notnull(df['Flight #'])]
#Delete all NaN Route
df = df[pd.notnull(df['Route'])]

In [15]:
df.isnull().sum()

Date                      0
Time                     44
Location                  0
Operator                  0
Flight #                  0
Route                     0
AC Type                   0
Registration              8
cn/ln                    14
Aboard                    5
Aboard Passangers         4
Aboard Crew               5
Fatalities                0
Fatalities Passangers     7
Fatalities Crew           8
Ground                    1
Summary                   0
Aboard_Check              5
dtype: int64

In [16]:
df = df.fillna(0)
df.isnull().sum()

Date                     0
Time                     0
Location                 0
Operator                 0
Flight #                 0
Route                    0
AC Type                  0
Registration             0
cn/ln                    0
Aboard                   0
Aboard Passangers        0
Aboard Crew              0
Fatalities               0
Fatalities Passangers    0
Fatalities Crew          0
Ground                   0
Summary                  0
Aboard_Check             0
dtype: int64

In [17]:
#split year and month from date
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
#Place these columns next to date
column_order = ['Date', 'Year', 'Month'] + [col for col in df.columns if col not in ['Date', 'Year', 'Month']]
df = df[column_order]
df['Fatality_Ratio (%)'] = (df['Fatalities'] / df['Aboard'])*100
df.head()

,Date,Year,Month,Time,Location,Operator,Flight #,Route,AC Type,Registration,...,Aboard,Aboard Passangers,Aboard Crew,Fatalities,Fatalities Passangers,Fatalities Crew,Ground,Summary,Aboard_Check,Fatality_Ratio (%)
141,1928-01-16,1928,1,0,"Solis, Uruguay",Aeropostale,F-AIKG,Buenos Aires - Montevideo,Latecoere 25,F-Aikg,...,2.0,0.0,2.0,2.0,0.0,2.0,0.0,The wing on the mail plane detached from the a...,2.0,100.000000
217,1930-01-19,1930,1,18:23,"Oceanside, California",Maddux Airlines,7,"Aqua Caliente, Mexico - Los Angeles",Ford 5-At-C Tri Motor,Nc9689,...,16.0,14.0,2.0,16.0,14.0,2.0,0.0,"While en route to Los Angeles, the pilot, flyi...",16.0,100.000000
247,1931-03-31,1931,3,10:45,"Bazaar, Kansas",Trans Continental And Western Air,599,Kansas City - Wichita - Los Angeles,Fokker F10A Trimotor,Nc-999,...,8.0,6.0,2.0,8.0,6.0,2.0,0.0,"Shortly after taking off from Kansas City, one...",8.0,100.000000
348,1934-08-31,1934,8,22:45,"Amazonia, Missouri",Rapid Air Transport,6,Kansas City - Saint Joseph - Omaha,Stinson Sm-6000B,Nc11118,...,5.0,4.0,1.0,5.0,4.0,1.0,0.0,The plane crashed about 11 miles from St. Jose...,5.0,100.000000
370,1935-05-06,1935,5,3:30,"Atlanta, Missouri",Trans Continental And Western Air,6,Los Angeles - Albuquerque - Kanasas City - Wa...,Douglas Dc-2-112,Nc13785,...,14.0,12.0,2.0,5.0,3.0,2.0,0.0,The plane crashed while en route from Albuquer...,14.0,35.714286


In [18]:
# Initialize the Nominatim geocoder
# kan tanya ms kenapa inda dapat

geolocator = Nominatim(user_agent="geoapiExercises")

# Function to geocode location name to coordinates
def geocode_location(location):
    try:
        loc = geolocator.geocode(location)
        return pd.Series([loc.latitude, loc.longitude])
    except:
        return pd.Series([None, None])

In [19]:
#Create Columns with coordinate of srash place
df[['Latitude CRASH PLACE', 'Longitude CRASH PLACE']] = df['Location'].apply(geocode_location)

In [20]:
#Split Data in Route
split_routes = df['Route'].str.split(' - ', n=1, expand=True)
df

,Date,Year,Month,Time,Location,Operator,Flight #,Route,AC Type,Registration,...,Aboard Crew,Fatalities,Fatalities Passangers,Fatalities Crew,Ground,Summary,Aboard_Check,Fatality_Ratio (%),Latitude CRASH PLACE,Longitude CRASH PLACE
141,1928-01-16,1928,1,0,"Solis, Uruguay",Aeropostale,F-AIKG,Buenos Aires - Montevideo,Latecoere 25,F-Aikg,...,2.0,2.0,0.0,2.0,0.0,The wing on the mail plane detached from the a...,2.0,100.000000,None,None
217,1930-01-19,1930,1,18:23,"Oceanside, California",Maddux Airlines,7,"Aqua Caliente, Mexico - Los Angeles",Ford 5-At-C Tri Motor,Nc9689,...,2.0,16.0,14.0,2.0,0.0,"While en route to Los Angeles, the pilot, flyi...",16.0,100.000000,None,None
247,1931-03-31,1931,3,10:45,"Bazaar, Kansas",Trans Continental And Western Air,599,Kansas City - Wichita - Los Angeles,Fokker F10A Trimotor,Nc-999,...,2.0,8.0,6.0,2.0,0.0,"Shortly after taking off from Kansas City, one...",8.0,100.000000,None,None
348,1934-08-31,1934,8,22:45,"Amazonia, Missouri",Rapid Air Transport,6,Kansas City - Saint Joseph - Omaha,Stinson Sm-6000B,Nc11118,...,1.0,5.0,4.0,1.0,0.0,The plane crashed about 11 miles from St. Jose...,5.0,100.000000,None,None
370,1935-05-06,1935,5,3:30,"Atlanta, Missouri",Trans Continental And Western Air,6,Los Angeles - Albuquerque - Kanasas City - Wa...,Douglas Dc-2-112,Nc13785,...,2.0,5.0,3.0,2.0,0.0,The plane crashed while en route from Albuquer...,14.0,35.714286,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4988,2022-03-21,2022,3,6:22,"Wuzhou, Guangxi, China",China Eastern Airlines,MU 5735,Kunming - Guangzhou,Boeing 737-89P Wl,B-1791,...,9.0,132.0,123.0,9.0,0.0,"Flying at 29,098 feet, the jetliner started de...",132.0,100.000000,None,None
4990,2022-07-16,2022,7,22:47,"Eleftheroupolis, Greece",Meridian,MEM3032,Nis- Amman,Antonov An-12,Ur-Cic,...,8.0,8.0,0.0,8.0,0.0,The cargo plane carrying eight crew members an...,8.0,100.000000,None,None
4991,2022-11-06,2022,11,8:53,"Bukoba, Tanzania",Precision Air,PW494,Dar Es-Salaam -Bukoba,Atr 42-500,5H-Pwf,...,4.0,19.0,17.0,2.0,0.0,"While on final approach to Bukoba Airport, the...",43.0,44.186047,None,None
4992,2022-11-18,2022,11,15:11,"Lima, Peru",Latam,LA2213,Lima - Juliaca,Airbus 320-271N,Cc-Bhb,...,6.0,0.0,0.0,0.0,0.0,The Airbus A320 collided with a fire truck whi...,108.0,0.000000,None,None


In [21]:
# Assign split data to new columns 'Source' and 'Destination'
df['Source'] = split_routes[0]
df['Destination'] = split_routes[1]

In [22]:
#Create Columns with coordinate of source and destination
df[['Latitude Source', 'Longitude Source']] = df['Source'].apply(geocode_location)
df[['Latitude Destination', 'Longitude Destination']] = df['Destination'].apply(geocode_location)

In [23]:
#Set Clear column order and save to csv
order=['Date','Year','Month','Time','Location','Latitude CRASH PLACE','Longitude CRASH PLACE','Operator','Flight #','Route','Source','Latitude Source', 'Longitude Source','Destination','Latitude Destination', 'Longitude Destination','AC Type','Registration','cn/ln','Aboard','Fatalities','Fatality_Ratio (%)','Summary']
df = df[order]
df.to_csv('EDA_Aircraft1.csv', index=False)